# Script to Retrieve Daily Night Light Image Data

## Methodology
### 1. Set Up the Conda Environment

To ensure all necessary libraries are installed and the environment is isolated, I created a new **Conda environment** specifically for this project.

- **Create the Conda Environment**:
  ```bash
  conda create --name viirs-env python=3.9

  
  conda activate viirs-env

  conda install geopandas rasterio
  pip install earthengine-api

  earthengine authenticate

- Note: A Google Cloud project will need to be created, and the Earth Engine API must be enabled in the Google Cloud Console.


In [1]:
import ee

# Initialize Earth Engine with the Project ID
ee.Initialize(project='viirs-data-collection')

print("Earth Engine API initialized successfully!")

Earth Engine API initialized successfully!


In [2]:
import geopandas as gpd

In [10]:
#Function to load shapefile and convert to Earth Engine geometry
def load_shapefile(shapefile_path):
    gdf = gpd.read_file(shapefile_path)
    geometry = gdf.geometry.union_all() # Merge all geometries into one if multiple features
    
    # Check if the geometry is valid
    if geometry.is_valid:
        # If it's a Polygon, return its exterior coordinates as a list
        if geometry.geom_type == 'Polygon':
            return ee.Geometry.Polygon([list(geometry.exterior.coords)])
        # If it's a MultiPolygon, handle it accordingly
        elif geometry.geom_type == 'MultiPolygon':
            coords = [list(polygon.exterior.coords) for polygon in geometry]
            return ee.Geometry.MultiPolygon(coords)
        else:
            raise ValueError("Unsupported geometry type")
    else:
        raise ValueError("Invalid geometry")

In [4]:
#set the path to your shapefile (update with your actual path)
shapefile_path = '/Users/kamiasalango/Desktop/Google_Earth/tofinoshapefile.shp' ## I created my own shapefile for Tofino 

In [7]:
#Load the shapefile into a GeoPandas DataFrame
gdf = gpd.read_file(shapefile_path)

#print the geometry to inspect it
print(gdf.geometry)

0    POLYGON ((-125.91207 49.15724, -125.77097 49.1...
Name: geometry, dtype: geometry


In [12]:
#Load the Area of Interest (AOI) from the shapefile
aoi = load_shapefile(shapefile_path)

#Print to check the AOI
print("AOI loaded:", aoi)

AOI loaded: ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -125.91207007223339,
              49.157243953787436
            ],
            [
              -125.77097231632538,
              49.10933079660669
            ],
            [
              -125.8715863279222,
              49.066247369452206
            ],
            [
              -125.91028402469024,
              49.130176762991624
            ],
            [
              -125.92308403208274,
              49.14731454331551
            ],
            [
              -125.91564216731966,
              49.15763330188023
            ],
            [
              -125.91207007223339,
              49.157243953787436
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})


In [35]:
#Collect daily VIIRS nightlight data
def collect_viirs_nightlights(aoi, start_date, end_date):
    # Use the VIIRS corrected radiances dataset
    viirs = ee.ImageCollection('NOAA/VIIRS/001/VNP46A1') \
            .filterDate(start_date, end_date) \
            .filterBounds(aoi)
    return viirs


## Google Earth Engine, Daily data set info: 
VNP46A1: VIIRS Daily Gridded Day Night Band 500m Linear Lat Lon Grid Night
https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_001_VNP46A1


In [29]:
#define date range
start_date = '2023-09-01'
end_date = '2023-09-30'

In [36]:
# Collect the nightlight data for last week
viirs_data = collect_viirs_nightlights(aoi, start_date, end_date)


In [37]:
num_images = viirs_data.size().getInfo()
print(f"Number of images retrieved: {num_images}")

Number of images retrieved: 29


In [38]:
first_image = viirs_data.first()
print(first_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'BrightnessTemperature_M12', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 163.8375}, 'crs': 'EPSG:4326', 'crs_transform': [0.00416666666, 0, -180, 0, -0.00416666666, 80]}, {'id': 'BrightnessTemperature_M13', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 163.8375}, 'crs': 'EPSG:4326', 'crs_transform': [0.00416666666, 0, -180, 0, -0.00416666666, 80]}, {'id': 'BrightnessTemperature_M15', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 268.69350000000003}, 'crs': 'EPSG:4326', 'crs_transform': [0.00416666666, 0, -180, 0, -0.00416666666, 80]}, {'id': 'BrightnessTemperature_M16', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 281.8005}, 'crs': 'EPSG:4326', 'crs_transform': [0.00416666666, 0, -180, 0, -0.00416666666, 80]}, {'id': 'DNB_At_Sensor_Radiance_500m', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 6553.5}, 'crs

## Export Images
- export images to google drive, images can then be downloaded locally
- alternatively export to google cloud storage
- google drive is specified ; google earth engine is linked to google account used. 


In [39]:
#Function to export VIIRS data to Google Drive
def export_viirs_to_drive(viirs_data, aoi, folder_name):
    images = viirs_data.toList(viirs_data.size())  # Convert collection to a list
    num_images = images.size().getInfo()
    
    for i in range(num_images):
        image = ee.Image(images.get(i))
        date = image.date().format('YYYY-MM-dd').getInfo()  # Extract date as string
        
        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=image.clip(aoi),  # Clip to the AOI (Area of Interest)
            description=f'VIIRS_{date}',  # Task description
            folder=folder_name,  # Folder in Google Drive
            fileNamePrefix=f'VIIRS_nightlight_{date}',  # File name prefix
            scale=500,  # Set the resolution in meters
            region=aoi.getInfo()['coordinates'],  # Define the export region (AOI)
            fileFormat='GeoTIFF'  # File format
        )
        task.start()
        print(f"Exporting VIIRS_nightlight_{date} to Google Drive...")

In [40]:
# Set the folder name where images will be stored in Google Drive
folder_name = 'VIIRS_Nightlight_Data'  # This folder will be created in Google Drive if it doesn't exist

In [41]:
#call the function to export the VIIRS data to Google Drive
export_viirs_to_drive(viirs_data, aoi, folder_name)

Exporting VIIRS_nightlight_2023-09-01 to Google Drive...
Exporting VIIRS_nightlight_2023-09-02 to Google Drive...
Exporting VIIRS_nightlight_2023-09-03 to Google Drive...
Exporting VIIRS_nightlight_2023-09-04 to Google Drive...
Exporting VIIRS_nightlight_2023-09-05 to Google Drive...
Exporting VIIRS_nightlight_2023-09-06 to Google Drive...
Exporting VIIRS_nightlight_2023-09-07 to Google Drive...
Exporting VIIRS_nightlight_2023-09-08 to Google Drive...
Exporting VIIRS_nightlight_2023-09-09 to Google Drive...
Exporting VIIRS_nightlight_2023-09-10 to Google Drive...
Exporting VIIRS_nightlight_2023-09-11 to Google Drive...
Exporting VIIRS_nightlight_2023-09-12 to Google Drive...
Exporting VIIRS_nightlight_2023-09-13 to Google Drive...
Exporting VIIRS_nightlight_2023-09-14 to Google Drive...
Exporting VIIRS_nightlight_2023-09-15 to Google Drive...
Exporting VIIRS_nightlight_2023-09-16 to Google Drive...
Exporting VIIRS_nightlight_2023-09-17 to Google Drive...
Exporting VIIRS_nightlight_2023

### Summary
- Shapefile Input: I provided an Area of Interest (AOI) defined by a shapefile, which specifies the geographical region for the data collection.
- Data Collection: Using Google Earth Engine's API, I accessed the NOAA/VIIRS/001/VNP46A1 dataset and collected daily nightlight data for the specified date range (e.g., Sep 1 to Sep 29, 2023).
- Exporting Data: The collected images were exported as GeoTIFF files to my Google Drive, where they can be downloaded and further analyzed locally.

### Auatomation

If the data were to be collected on a daily basis in real time the script could be automated: 
- Dynamic Date Handling: Instead of specifying a fixed date range, the script can be modified to dynamically collect the previous night's data by using Python’s datetime module. For example:
  Each time the script runs, it can use the current date to fetch the data from the previous night.
  The script could be scheduled to run daily using a scheduling tool like cron (on macOS/Linux) or Task Scheduler (on Windows).
- Scheduling the Script: The process can be automated by scheduling the script to run at a specific time each day, ensuring that the latest nightlight data is always retrieved and exported. This can be achieved using:
   cron: A job can be set up to run the script every day at a designated time.
   Python Scheduler: Libraries like schedule or APScheduler can be used to run the script on a daily basis within Python itself.
- Continuous Data Collection: By automating this process, the script will continuously retrieve and export nightlight data daily, allowing for continuous monitoring and analysis of nightlight changes in the area of interest.
